In [11]:
!git clone https://github.com/YoongiKim/CIFAR-10-images

fatal: destination path 'CIFAR-10-images' already exists and is not an empty directory.


In [2]:
#activating CUDA
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-twtjcgjb
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-twtjcgjb
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=3cca6b413b390445ed08345a8f3c6fd0329e4da2a2de7f20676f2a8bfcbd0d35
  Stored in directory: /tmp/pip-ephem-wheel-cache-j2tycmx2/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [32]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import datasets,transforms
import torch.nn as nn
import torch
import cv2


In [16]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')


CUDA is available!  Training on GPU ...


In [27]:
import os
os.listdir('/content/CIFAR-10-images/train')
#############
print('==> Preparing data..')

classes = os.listdir('/content/CIFAR-10-images/train')

==> Preparing data..


In [29]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
trainset = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [30]:
def load_images(train_path, test_path):
  train_classes = os.listdir(train_path)
  test_classes = os.listdir(test_path)

  train_images = []
  test_images = []

  print("Loading Train Images")
  for cls in train_classes:
    print("Reading images of class: ", cls)
    cls_path = os.path.join(train_path, cls)
    for img_id in os.listdir(cls_path):
      img = cv2.imread(os.path.join(cls_path, img_id))
      train_images.append([img, cls])

  print("Loading Test Images")
  for cls in test_classes:
    print("Reading images of class: ", cls)
    cls_path = os.path.join(test_path, cls)
    for img_id in os.listdir(cls_path):
      img = cv2.imread(os.path.join(cls_path, img_id))
      test_images.append([img, cls])
  
  return train_images, test_images


In [33]:
train_path = '/content/CIFAR-10-images/train'
test_path = '/content/CIFAR-10-images/test'

train_images, test_images = load_images(train_path, test_path)
print(len(train_images), len(test_images))

Loading Train Images
Reading images of class:  bird
Reading images of class:  cat
Reading images of class:  dog
Reading images of class:  automobile
Reading images of class:  horse
Reading images of class:  frog
Reading images of class:  ship
Reading images of class:  deer
Reading images of class:  airplane
Reading images of class:  truck
Loading Test Images
Reading images of class:  bird
Reading images of class:  cat
Reading images of class:  dog
Reading images of class:  automobile
Reading images of class:  horse
Reading images of class:  frog
Reading images of class:  ship
Reading images of class:  deer
Reading images of class:  airplane
Reading images of class:  truck
50000 10000


In [ ]:
class CifarDatasetT(Dataset):
    def __init__(self, train_images, transform=None):
        '''
        Args : 
            csvfile : train/val/test csvfiles
            audio_dir : directory that contains folders 0 - f
        '''
        self.train_images = train_images
        self.transform = transform

    # get one segment (==59049 samples) and its 50-d label
    def __getitem__(self, index):

      if self.transform:
        x = torch.tensor(self.transform(Image.fromarray(self.train_images[index][0]))).to(torch.float32).unsqueeze(0)
      else:
        x = torch.tensor(Image.fromarray(self.train_images[index][0])).to(torch.float32).unsqueeze(0)
      y = self.train_images[index][1]

      sample = {'image': x, 'class': y}

      # if self.transform:
      #   sample = self.transform(sample)


      return sample